In [107]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [108]:
# Carregar o dataset
df = pd.read_csv('raw_data/artDataset.csv')
# Verificar as primeiras linhas
display(df.head())
df.info()

,Unnamed: 0,price,artist,title,yearCreation,signed,condition,period,movement
0,0,28.500 USD,Tommaso Ottieri,Bayreuth Opera,2021,Signed on verso,This work is in excellent condition.,Contemporary,Baroque
1,1,3.000 USD,Pavel Tchelitchew,Drawings of the Opera,First Half 20th Century,Signed and titled,Not examined out of frame.No obvious signs of ...,Post-War,Surrealism
2,2,5.000 USD,Leo Gabin,Two on Sidewalk,2016,"Signed, titled and dated on verso",This work is in excellent condition.,Contemporary,Abstract
3,3,5.000 USD,Matthias Dornfeld,Blumenszene,2010,"Signed, titled and dated on the reverse with t...",This work is in excellent condition.There is m...,Contemporary,Abstract
4,4,2.500 USD,Alexis Marguerite Teplin,Feverish Embarkation,2001,Signed on verso,This work is in excellent condition.,Contemporary,Abstract


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    754 non-null    int64 
 1   price         754 non-null    object
 2   artist        753 non-null    object
 3   title         754 non-null    object
 4   yearCreation  754 non-null    object
 5   signed        754 non-null    object
 6   condition     754 non-null    object
 7   period        754 non-null    object
 8   movement      754 non-null    object
dtypes: int64(1), object(8)
memory usage: 53.1+ KB


In [109]:
rows_with_null = df[df.isnull().any(axis=1)]
rows_with_null

,Unnamed: 0,price,artist,title,yearCreation,signed,condition,period,movement
725,725,1.275 USD,NaN,[nan],[nan],Signed and dated in pencil to verso,Not examined out of frame.Minor sheet undulati...,Contemporary,Realism


In [110]:
df = df.drop(725)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 753 entries, 0 to 753
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    753 non-null    int64 
 1   price         753 non-null    object
 2   artist        753 non-null    object
 3   title         753 non-null    object
 4   yearCreation  753 non-null    object
 5   signed        753 non-null    object
 6   condition     753 non-null    object
 7   period        753 non-null    object
 8   movement      753 non-null    object
dtypes: int64(1), object(8)
memory usage: 58.8+ KB


In [111]:
# Converter price strings em integers
df['price'] = df['price'].str.replace('USD', '')
df['price'] = df['price'].str.replace('.', '')
df['price'] = df['price'].astype(int)
df.head()

,Unnamed: 0,price,artist,title,yearCreation,signed,condition,period,movement
0,0,28500,Tommaso Ottieri,Bayreuth Opera,2021,Signed on verso,This work is in excellent condition.,Contemporary,Baroque
1,1,3000,Pavel Tchelitchew,Drawings of the Opera,First Half 20th Century,Signed and titled,Not examined out of frame.No obvious signs of ...,Post-War,Surrealism
2,2,5000,Leo Gabin,Two on Sidewalk,2016,"Signed, titled and dated on verso",This work is in excellent condition.,Contemporary,Abstract
3,3,5000,Matthias Dornfeld,Blumenszene,2010,"Signed, titled and dated on the reverse with t...",This work is in excellent condition.There is m...,Contemporary,Abstract
4,4,2500,Alexis Marguerite Teplin,Feverish Embarkation,2001,Signed on verso,This work is in excellent condition.,Contemporary,Abstract


In [112]:
df['yearCreation'].unique()

array(['2021', 'First Half 20th Century ', '2016', '2010', '2001', '1977',
       '2007', '1993', '2014', '1973', '[nan]', '2008', '2015', '2012',
       '2002', '1992', '2020', '1992 - 2004', '1991', '2018', '1920',
       'Mid 20th Century ', 'Second Half 20th Century ', '2003', '2000',
       '2005', '1999', '1958', '19th Century ', 'Circa 1970 - 1979',
       '1877', 'Circa 1877', 'Second Half 19th Century ', '2004', '2013',
       '2006', '1990', 'Circa 1925', '1935', '1928', '2022',
       '1961, printed in 2010', '1994',
       '3D printed using ABS, PLA plastics, resin, automobile paints, etched brass, dry transfers, acrylic mirror, batik fabric, quartz clock',
       '1992 - 1993', '1996', '2017', '2019', '2011', '1948', '1971',
       '1960', '1972', '1984', 'Late 20th Century ', '1981', '1976',
       '1950', '1896', '2002 - 2020', '1965 - 2018', '1975', '1968',
       '1970', '1969', '1998', '1996-2003', '1940', '1988',
       'Late 19th Century ', '1892', '1947', '1995', '

In [113]:
# Converter strings de yearCreation em integers

# Remover 'circa'
df['yearCreation'] = df['yearCreation'].str.replace('circa', '')
# Remover 'Circa'
df['yearCreation'] = df['yearCreation'].str.replace('Circa', '')
# Strip empty spaces
df['yearCreation'] = df['yearCreation'].str.strip()

In [114]:
def clean_year(year_text):
    if pd.isna(year_text) or str(year_text).strip() in ['', '[nan]', 'nan']:
        return None

    year_text = str(year_text).strip()

    # Caso 1: Ano exato (ex.: "2021")
    match_exact = re.search(r'^(\d{4})(?:\.0)?$', year_text)
    if match_exact:
        return int(match_exact.group(1))

    # Caso 2: Períodos do século 20 (ex.: "First Half 20th Century")
    match_20th = re.search(r'(First Half|Second Half|Mid|Late|Early)\s*20th Century', year_text, re.IGNORECASE)
    if match_20th:
        period = match_20th.group(1).lower()
        base_year = 1900
        if 'first' in period:
            return base_year + 25
        elif 'second' in period:
            return base_year + 75
        elif 'mid' in period:
            return base_year + 50
        elif 'late' in period:
            return base_year + 80
        elif 'early' in period:
            return base_year + 20

    # Caso 3: Intervalos (ex.: "1992 - 2004")
    match_range = re.search(r'(\d{4})\s*[-–]\s*(\d{4})', year_text)
    if match_range:
        start, end = map(int, match_range.groups())
        return round((start + end) / 2)

    # Caso 4: Décadas (ex.: "1930 - 1939")
    match_decade = re.search(r'(\d{3})0\s*-\s*(\d{3})9', year_text)
    if match_decade:
        return int(match_decade.group(1)) + 5

    # Caso 5: Século genérico (ex.: "20th Century")
    match_century = re.search(r'(\d{1,2})(?:st|nd|rd|th)\s*Century', year_text, re.IGNORECASE)
    if match_century:
        century = int(match_century.group(1))
        return (century - 1) * 100 + 50

    # Caso 6: Valores complexos (ex.: "1961, printed in 2010")
    match_complex = re.search(r'(\d{4})', year_text)
    if match_complex:
        return int(match_complex.group(1))

    return None

# Aplicar a função
df['yearCreation'] = df['yearCreation'].apply(clean_year).astype('Int64')

# Verificação
print("Valores nulos:", df['yearCreation'].isna().sum())
print("\nDistribuição dos anos:")
print(df['yearCreation'].value_counts().head(10))


Valores nulos: 19

Distribuição dos anos:
yearCreation
2012    34
1975    30
1990    28
1989    24
2008    23
1986    21
2021    19
2011    19
2016    18
2001    18
Name: count, dtype: Int64


In [115]:
# 1. Calcular a média dos anos (ignorando os nulos)
year_mean = int(df['yearCreation'].mean())

# 2. Substituir os valores nulos pela média (convertida para inteiro)
df['yearCreation'] = df['yearCreation'].fillna(year_mean).astype(int)

# 3. Verificação
print(f"Média dos anos utilizada: {year_mean}")
print("\nDistribuição após substituição:")
print(df['yearCreation'].value_counts().head())

Média dos anos utilizada: 1989

Distribuição após substituição:
yearCreation
1989    43
2012    34
1975    30
1990    28
2008    23
Name: count, dtype: int64


In [116]:
df['yearCreation'].unique()

array([2021, 1925, 2016, 2010, 2001, 1977, 2007, 1993, 2014, 1973, 1989,
       2008, 2015, 2012, 2002, 1992, 2020, 1998, 1991, 2018, 1920, 1950,
       1975, 2003, 2000, 2005, 1999, 1958, 1850, 1974, 1877, 2004, 2013,
       2006, 1990, 1935, 1928, 2022, 1961, 1994, 1996, 2017, 2019, 2011,
       1948, 1971, 1960, 1972, 1984, 1980, 1981, 1976, 1896, 1968, 1970,
       1969, 1940, 1988, 1892, 1947, 1995, 1997, 1983, 1964, 1959, 1965,
       1939, 1952, 1930, 1949, 1978, 1955, 1957, 2009, 1962, 1986, 1985,
       1900, 1967, 1934, 1894, 1941, 1936, 1953, 1943, 1946, 1937, 1944,
       1913, 1987, 1979, 2050, 1982, 1931, 1954])

In [117]:
# Criar coluna binária
df['is_signed'] = df['signed'].notna().astype(int)  # 1 se há assinatura, 0 se NaN

# Opcional: Extrair detalhes específicos (ex.: 'numerado')
df['is_numbered'] = df['signed'].str.contains('numbered', case=False, na=False).astype(int)

In [118]:
# Definir regras para o score baseado em palavras-chave
conditions = [
    (df['condition'].str.contains('excellent|perfect|mint|pristine|as new', case=False, na=False)),  # Score 3
    (df['condition'].str.contains('very good|good|fine|sound|stable', case=False, na=False)),  # Score 2
    (df['condition'].str.contains('regular|fair|acceptable|showing age|age appropriate', case=False, na=False)),  # Score 1
    (df['condition'].str.contains('poor|bad|damaged|worn|faded', case=False, na=False))  # Score 0
]

# Criar a coluna 'condition_score' com np.select
scores = [3, 2, 1, 0]  # Valores para cada condição acima
df['condition_score'] = np.select(conditions, scores, default=1)  # default=1 assume "regular" se não encontrar nada

# Verificar resultados
print(df[['condition', 'condition_score']].sample(10))  # Checar amostras aleatórias

# Ver distribuição
print(df['condition_score'].value_counts())

                                             condition  condition_score
160                               Excellent condition.                3
571  This work is in very good condition.No obvious...                2
181                The work is in excellent condition.                3
275                               Excellent condition.                3
191  The canvas is relined. Very fine pigment separ...                2
67                This work is in excellent condition.                3
130                               Excellent condition.                3
176                The work is in excellent condition.                3
7    This work is in very good condition, direct fr...                2
285                               Excellent condition.                3
condition_score
2    296
3    278
1    179
Name: count, dtype: int64


In [119]:
df['movement'].value_counts(dropna=False)

movement
Realism                                  176
Abstract                                 153
Expressionism                            103
Pop Art                                   88
Conceptual                                73
Surrealism                                21
Impressionism                             20
Geometric Abstraction                     19
Minimalism                                18
Abstract Expressionism                    16
Feminist Art                               7
Traditional                                5
Organic/Biomorphic Abstraction             5
Nouveau Réalisme                           4
[nan]                                      4
Post-Minimalism                            4
Post-Impressionism                         4
Social Realism                             4
Photorealism                               4
Modernism                                  3
Performance Art                            3
Street Art                                 3
E

In [120]:
# 1. Preencher NaN com moda (Realism)
df['movement'] = df['movement'].fillna('Realism')

# 2. Agrupar movimentos raros (menos de 10 ocorrências)
movement_counts = df['movement'].value_counts()
df['movement_clean'] = df['movement'].where(
    df['movement'].map(movement_counts) >= 10,
    'Other'
)

In [121]:
# Renomear a primeira coluna
df = df.rename(columns={'Unnamed: 0': 'index'})

# Definir as colunas a serem removidas
cols_to_drop = ['artist', 'title', 'signed', 'condition', 'period', 'movement', 'is_signed']

# Remover as colunas especificadas
df = df.drop(columns=cols_to_drop)

In [122]:
display(df.head())

,index,price,yearCreation,is_numbered,condition_score,movement_clean
0,0,28500,2021,0,3,Other
1,1,3000,1925,0,1,Surrealism
2,2,5000,2016,0,3,Abstract
3,3,5000,2010,0,3,Abstract
4,4,2500,2001,0,3,Abstract


In [123]:
df_clean = df.copy()

output_path = 'clean_dataset_final/art_prices_clean.csv'
df_clean.to_csv(output_path, index=False)

print(f"\nDataset salvo com sucesso em: {output_path}")
print(f"Shape final: {df_clean.shape}")


Dataset salvo com sucesso em: clean_dataset_final/art_prices_clean.csv
Shape final: (753, 6)
